<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD Tutorial

FAST-OAD is a framework for performing rapid Overall Aircraft Design. The computational core of FAST-OAD is based on the  [OpenMDAO framework](https://openmdao.org/).

## 1. Setting up and analyzing the initial problem

To organize our work, we propose to use two user folders `data/` and `workdir/`. For instance, in `data/` we store a XML file which describes the [CeRAS reference case](http://ceras.ilr.rwth-aachen.de/trac/wiki/CeRAS/AircraftDesigns/CSR01). In `workdir/`, we store files generated or modified by FAST-OAD.

In [1]:
import os.path as pth
import openmdao.api as om
import logging
import shutil
import fastoad.api as oad


DATA_FOLDER_PATH = "data"

WORK_FOLDER_PATH = "workdir"

CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, "oad_process.yml")
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "CeRAS01_baseline.xml")

# For having log messages on screen
logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

After defining a configuration file name, we can ask FAST-OAD to generate a default configuration file based on the default OAD model implemented in the framework:

In [2]:
oad.generate_configuration_file(
    CONFIGURATION_FILE, overwrite=True, distribution_name="fast-oad-cs25"
)

# The above generated configuration uses the quick and simple Breguet module to compute performances.
# If you want to use the more accurate, much more CPU-costly time-step integration, uncomment and run
# the next line:
# shutil.copy(pth.join(DATA_FOLDER_PATH, "oad_process_timestep_mission.yml"), CONFIGURATION_FILE)

INFO    : Loading FAST-OAD plugin bundled
INFO    : Loading bundles from fastoad.models
INFO    : Installed bundle fastoad.models.performances.mission.segments.registered.start (ID 41 )
INFO    : Installed bundle fastoad.models.__init__ (ID 54 )
INFO    : Installed bundle fastoad.models.performances.mission.exceptions (ID 11 )
INFO    : Installed bundle fastoad.models.performances.mission.openmdao.payload_range (ID 27 )
INFO    : Installed bundle fastoad.models.performances.mission.openmdao.mission_run (ID 25 )
INFO    : Installed bundle fastoad.models.performances.mission.routes (ID 32 )
INFO    : Installed bundle fastoad.models.performances.mission.segments.registered.takeoff.end_of_takeoff (ID 42 )
INFO    : Installed bundle fastoad.models.performances.mission.segments.registered.takeoff.rotation (ID 43 )
INFO    : Installed bundle fastoad.models.performances.mission.openmdao.mission_wrapper (ID 26 )
INFO    : Installed bundle fastoad.models.performances.mission.polar_modifier (ID 3

'C:\\Adam\\SUPAERO\\STAGE RECHERCHE\\FAST\\FAST-OAD_CS25\\FAST-OAD_notebooks\\fast-oad-cs25\\01_tutorial\\workdir\\oad_process.yml'

You can now checkout the generated [configuration file](./workdir/oad_process.yml).

In this configuration file, we have specified an input file name 'problem_inputs.xml'. We can ask FAST-OAD to generate the inputs of the default model with the CeRAS parameters as default values:

In [ ]:
oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

You can now checkout the generated [input file](./workdir/problem_inputs.xml). As shown previously in the user file architecture, the values in this file can be modified by the user and will be considered by FAST-OAD when executing a computational process.

A useful feature that FAST-OAD provides is to list the outputs of the model defined in the configuration file:

In [ ]:
oad.list_variables(CONFIGURATION_FILE)

Another useful feature is to list the modules of the model defined in the configuration file:

In [ ]:
oad.list_modules(CONFIGURATION_FILE)
# api.list_modules(CONFIGURATION_FILE, verbose=True) # Use this line instead of the previous one for detailed information (text format only)

Another useful feature is the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) visualization available in OpenMDAO to see the structure of the model:

In [ ]:
N2_FILE = pth.join(WORK_FOLDER_PATH, "n2.html")
oad.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame

IFrame(src=N2_FILE, width="100%", height="500px")

Alternatively, you can use [WhatsOpt](https://github.com/OneraHub/WhatsOpt-Doc#whatsopt-documentation) as a web service to generate the XDSM of the problem.

*This command requires internet access and can take some time (~ 1 minute on the first run), so it is currently deactivated, but feel free to uncomment these lines.*

In [ ]:
# XDSM_FILE = pth.join(WORK_FOLDER_PATH, 'xdsm.html')
# oad.write_xdsm(CONFIGURATION_FILE, XDSM_FILE, overwrite=True)
# from IPython.display import IFrame
# IFrame(src=XDSM_FILE, width='100%', height='500px')

## 2. Running your first MDA

### CeRAS
Here we run an MDA, that is solving the multidisciplinary couplings using the different nested solvers in the model, without running the optimization problem even if it is defined in the configuration file.

*(This run should take around 10 seconds)*

In [ ]:
eval_problem = oad.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

> _**Note**: the warning you see at end of computation is normal. FAST-OAD allows to define validity domain for each model, and the propulsion model has been validated for a certain range of turbine inlet temperature (a.k.a. T4). The T4 value we use is slightly outside the validated range, hence the warning. The check of validity domain issues warnings as soon as a value is outside the bounds, but in the end, it's up to the user to decide if the problem is real or not, though it may require a certain knowledge about the used models.  
> In this case, the model has been validated for T4 between 1400K and 1600K and using it for a value of 1633K can be considered as OK._

Let's save these results. We will use them in the next notebook, that shows some post-processing utilities.

In [ ]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs.xml")
CeRAS_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs_CeRAS_2750nm_mda.xml")
shutil.copy(OUTPUT_FILE, CeRAS_OUTPUT_FILE)
MISSION_FILE = pth.join(WORK_FOLDER_PATH, "flight_points.csv")
CeRAS_MISSION_FILE = pth.join(WORK_FOLDER_PATH, "mission_CeRAS_2750nm_mda.csv")
shutil.copy(MISSION_FILE, CeRAS_MISSION_FILE)

The `variable-viewer` provides a way to inspect the content of the XML file. The dropdown lists above the table allow to filter the displayed variable.

In [ ]:
oad.variable_viewer(OUTPUT_FILE)

### CeRAS for 800 nm range
Here we run an MDA but we change one of the Top Level Aircraft Requirement (TLAR): the range. We choose a 800 nm range instead of the 2750 nm of the CeRAS. For that we use the `VariableViewer` tool on the input file to change the range (do not forget to save!). Just like this:
![variable_viewer](./img/variable_viewer_change_range.gif)

In [ ]:
INPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_inputs.xml")
oad.variable_viewer(INPUT_FILE)

You can also do this operation through Python like this:

In [ ]:
input_data = oad.DataFile(INPUT_FILE)
input_data["data:TLAR:range"].value = 800.0
input_data.save()

Now that the range has been changed, we run again the MDA.

*(This run should take around 10 seconds)*

In [ ]:
eval_problem = oad.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

Let's save again these new results, for post-processing them in next notebook.

In [ ]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs.xml")
CeRAS_800nm_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs_CeRAS_800nm_mda.xml")
shutil.copy(OUTPUT_FILE, CeRAS_800nm_OUTPUT_FILE)
MISSION_FILE = pth.join(WORK_FOLDER_PATH, "flight_points.csv")
CeRAS_800nm_MISSION_FILE = pth.join(WORK_FOLDER_PATH, "mission_CeRAS_800nm_mda.csv")
shutil.copy(MISSION_FILE, CeRAS_800nm_MISSION_FILE)

## 3. Running your first MDO

## CeRAS 2750 nm optimization

It is possible to run an optimization problem on top of your analysis. Thus, you can choose to set some global inputs as design variables, and global outputs as constraints or the objective. Here is an example given in the default configuration file that aims at:
- minimizing the fuel consumption for the mission (objective),
- with respect to the wing aspect ratio (design variables),
- subject to a wing span limit (constraints).

The last part of the configuration file .yaml is where this optimization problem is defined:

```yaml
optimization:  # This section is needed only if optimization process is run
  design_variables:
    - name: data:geometry:wing:aspect_ratio
      lower: 9.0
      upper: 18.0
  constraints:
    - name: data:geometry:wing:span
      upper: 60.0
  objective:
    - name: data:mission:sizing:needed_block_fuel
      scaler: 1.e-4
```

Note that we need to use [scaling](https://openmdao.org/twodocs/versions/latest/features/core_features/defining_components/scaling.html) on the objective (fuel mass for the mission) so that it has the same order of magnitude than the design variable (wing aspect ratio) by adding the line `scaler = 1e-4`. Without this the optimizer might not detect that the steps it makes on the wing aspect ratio has an effect on decreasing the fuel mass and could result in an early stop.


*(This run should take around 3 minutes)*

In [ ]:
# Set back the inputs from the reference CeRAS 2750 nm
oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

To visualize and edit the optimization problem definition (present in the configuration file .yml) you can use the `optimization_viewer` tool. If design variables or constraints have active bounds they are yellow whereas they are red if they are violated. Modifiying the `Initial Value` will modify the input file defined in the configuration file .yml whereas `Value` corresponds to the value found in the output file defined in the configuration file (here it is the 800 nm MDA run).

In [ ]:
oad.optimization_viewer(CONFIGURATION_FILE)

In [ ]:
optim_problem = oad.optimize_problem(CONFIGURATION_FILE, overwrite=True)

Let's save these results:

In [ ]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs.xml")
CeRAS_OPT_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs_CeRAS_2750nm_mdo.xml")
shutil.copy(OUTPUT_FILE, CeRAS_OPT_OUTPUT_FILE)
MISSION_FILE = pth.join(WORK_FOLDER_PATH, "flight_points.csv")
CeRAS_OPT_MISSION_FILE = pth.join(WORK_FOLDER_PATH, "mission_CeRAS_2750nm_mdo.csv")
shutil.copy(MISSION_FILE, CeRAS_OPT_MISSION_FILE)

The `optimizer_viewer` offers a convenient summary of the optimization result:

In [ ]:
oad.optimization_viewer(CONFIGURATION_FILE)

We see that the optimizer finds a compromise between a smaller induced drag (higher aspect ratio) and a heavier wing.

**Note: Finding such compromise on real aircraft is still a subject of research. Please take this result as informational.**

You can use the `VariableViewer` tool to see the optimization results for all variables of the system by loading the .xml output file:

In [ ]:
RESULT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs.xml")

In [ ]:
oad.variable_viewer(RESULT_FILE)